### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
성능 개선을 위한 테스트에서는 전체를 대상으로 인덱싱하지 않고, 5개 인덱싱 코드 실행 권장(주석 해제)

In [1]:
from preprocess.pp_basic import docs, BASE_DIR, EVAL_DIR, GOLD_EVIDENCE_CSV, GOLD_FIELDS_JSONL, RAW_DIR
import preprocess.pp_v4 as pp

/home/ohs3201/codeit/codeit-part3-team4/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import unicodedata
import json
import re
import numpy as np
import pandas as pd
import faiss
import torch

In [3]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

GEN_MODEL_MAIN = "gpt-5-mini"
GEN_MODEL_AUX  = "gpt-5-nano"

In [4]:
# 임베딩 및 인덱스 만들기
embed_model_name = "BAAI/bge-m3"
embed_model = SentenceTransformer(embed_model_name)

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 977.20it/s, Materializing param=pooler.dense.weight]                                


### 평가 데이터 로드

In [5]:
import pandas as pd

# questions.csv 로드
def load_questions_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(
        path,
        encoding="utf-8",
        quotechar='"',      # 따옴표 처리
        escapechar="\\",    # 혹시 이스케이프가 있으면
        engine="python",    # 복잡한 CSV에서 더 관대
    )
    # 컬럼명 확인/정리
    expected = ["instance_id","qid","doc_id","question","type"]
    assert all(c in df.columns for c in expected), df.columns.tolist()

    # 공백/NaN 정리
    df["doc_id"] = df["doc_id"].astype(str).str.strip()
    df["question"] = df["question"].astype(str).str.strip()
    df["type"] = df["type"].astype(str).str.strip()
    df["instance_id"] = df["instance_id"].astype(str).str.strip()
    df["qid"] = df["qid"].astype(str).str.strip()
    return df[expected]

# gold_fields.jsonl 로드
import json

def gold_to_text(v) -> str:
    if v is None:
        return ""
    if isinstance(v, str):
        return v.strip()
    # list/dict는 JSON 문자열로 고정
    return json.dumps(v, ensure_ascii=False)

def load_gold_fields_jsonl(path: Path) -> pd.DataFrame:
    out = []
    with path.open("r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            r = json.loads(line)
            iid = str(r.get("instance_id","")).strip()
            doc_id = str(r.get("doc_id","")).strip()
            fields = r.get("fields") or {}
            for k, v in fields.items():
                out.append({
                    "instance_id": iid,
                    "doc_id": doc_id,
                    "field": str(k).strip(),
                    "gold": gold_to_text(v),   # 항상 str
                })
    return pd.DataFrame(out)

def build_gold_anchor_map(df: pd.DataFrame):
    m = {}
    for _, r in df.iterrows():
        doc_id = str(r["doc_id"]).strip()
        iid = str(r["instance_id"]).strip()
        a = str(r.get("anchor_text","") or "").strip()

        if not a or a.upper() == "NOT_FOUND":
            continue

        key = (doc_id, iid)
        m.setdefault(key, set()).add(a)

    return {k: sorted(v) for k, v in m.items()}

### 로드

In [6]:
QUESTIONS_CSV = EVAL_DIR / "questions.csv"

questions_df = load_questions_csv(QUESTIONS_CSV)
gold_fields_df = load_gold_fields_jsonl(GOLD_FIELDS_JSONL)

gold_evidence_df = pd.read_csv(GOLD_EVIDENCE_CSV, encoding="utf-8")
GOLD_ANCHOR = build_gold_anchor_map(gold_evidence_df)

print("loaded:",
      "questions_df", len(questions_df),
      "gold_fields_df", len(gold_fields_df),
      "gold_evidence_df", len(gold_evidence_df),
      "anchors", len(GOLD_ANCHOR))

loaded: questions_df 311 gold_fields_df 630 gold_evidence_df 630 anchors 609


### doc_path 선택 함수

In [7]:
def pick_doc_path_by_name(docs: list[Path], doc_name: str) -> Path:
    target = unicodedata.normalize("NFC", str(doc_name)).strip()
    for p in docs:
        if unicodedata.normalize("NFC", p.name).strip() == target:
            return p
    raise FileNotFoundError(f"PDF not found in RAW_DIR: {doc_name}")

# 테스트용: gold_fields에 있는 문서 하나 골라서 doc_path 만들기
doc_name = gold_fields_df["doc_id"].iloc[0]
doc_path = pick_doc_path_by_name(docs, doc_name)

print("doc_path:", doc_path)
print("doc_name:", doc_path.name)

doc_path: /home/ohs3201/codeit/codeit-part3-team4/data/raw/files/(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
doc_name: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf


### pp로 청킹/인덱스 만들기

In [8]:
index, chunks = pp.gen_input(doc_path, embed_model)
print("n_chunks:", len(chunks))
print("chunk0:", chunks[0][:200])

n_chunks: 130
chunk0: 용역사업
- (복수의결권주식, 스톡옵션, 성과조건부주식) -
 
2024. 03.
목 차
1. 추진개요 3
2. 추진방안 5
3. 추진내용 9
4. 제안요청내용 24
5. 입찰관련사항 78
6. 제안서작성요령 82
7. 별지서식 및 붙임 94
Ⅰ. 추진개요
1 추진배경 및 방향
□ 「벤처기업육성에 관한 특별조치법」(이하 ‘벤처기업법’) 복수의결권주식, 스톡


### BM25 + retrieve_indices (R1/R2/R3) “새로 정의”

In [9]:
def tokenize(text: str):
    return re.findall(r"[A-Za-z0-9가-힣]+", (text or "").lower())

def build_bm25(chunks: list[str]):
    return BM25Okapi([tokenize(c) for c in chunks])

bm25 = build_bm25(chunks)


def retrieve_indices(
    retriever: str,
    query_text: str,
    chunks: list[str],
    index,
    embed_model,
    bm25,
    top_k: int = 20,
    hybrid_cand_k: int = 200 ,
):
    retriever = retriever.lower().strip()
    hybrid_cand_k = min(hybrid_cand_k, len(chunks))

    if retriever == "bm25":
        scores = bm25.get_scores(tokenize(query_text))
        idxs = np.argsort(scores)[::-1][:top_k]
        return [int(i) for i in idxs]

    if retriever == "dense":
        # sentence-transformers: encode -> np array
        q_emb = embed_model.encode([query_text])
        _, I = index.search(q_emb.astype("float32"), top_k)
        return [int(i) for i in I[0]]

    if retriever == "hybrid":
        # 1) BM25 후보
        scores = bm25.get_scores(tokenize(query_text))
        cand = np.argsort(scores)[::-1][:hybrid_cand_k]
        cand = [int(i) for i in cand]

        # 2) Dense 재랭킹 (cosine)
        cand_chunks = [chunks[i] for i in cand]
        cand_emb = embed_model.encode(cand_chunks)
        q_emb = embed_model.encode([query_text])

        cand_emb = cand_emb / (np.linalg.norm(cand_emb, axis=1, keepdims=True) + 1e-8)
        q_emb = q_emb / (np.linalg.norm(q_emb, axis=1, keepdims=True) + 1e-8)

        sims = (cand_emb @ q_emb.T).ravel()
        rerank = [cand[i] for i in np.argsort(sims)[::-1][:top_k]]
        return rerank

    raise ValueError(f"Unknown retriever: {retriever}")

### pp_v4로 chunks 뽑기

In [10]:
index, chunks = pp.gen_input(doc_path, embed_model)
print("n_chunks:", len(chunks))
print("first_len:", len(chunks[0]) if chunks else None)

n_chunks: 130
first_len: 800


### 길이 분포 확인

In [11]:
lens = [len(c) for c in chunks]
print("min/median/mean/max:",
      min(lens), sorted(lens)[len(lens)//2], sum(lens)/len(lens), max(lens))

# 너무 긴 청크 상위 5개
top = sorted(enumerate(lens), key=lambda x: x[1], reverse=True)[:5]
for i, L in top:
    print("\n--- chunk", i, "len", L, "---")
    print(chunks[i][:500])

min/median/mean/max: 751 800 799.623076923077 800

--- chunk 0 len 800 ---
용역사업
- (복수의결권주식, 스톡옵션, 성과조건부주식) -
 
2024. 03.
목 차
1. 추진개요 3
2. 추진방안 5
3. 추진내용 9
4. 제안요청내용 24
5. 입찰관련사항 78
6. 제안서작성요령 82
7. 별지서식 및 붙임 94
Ⅰ. 추진개요
1 추진배경 및 방향
□ 「벤처기업육성에 관한 특별조치법」(이하 ‘벤처기업법’) 복수의결권주식, 스톡옵
션(주식매수선택권), 성과조건부주식교부계약(RS) 등의 기능 고도화 및 이관,
신규 구축업무를 本 과업에서 추진
⚬ (복수의결권주식*) 벤처기업법 제16조의11에 따라 발행된 복수의결권주
식 보고 업무처리 시스템 구축
* 복수의결권이란? 모든 주주는 1주당 하나의 의결권을 갖는 주주평등원칙(상법)과 별도로
비상장 벤처기업 창업자에게만 1주당 최대 10배의 의결권 행사를 부여하는 제도
⚬ (스톡옵션*) 벤처기업법 제16조의3에 따라 부여된 벤처기업 스톡옵션
부여, 취소·철회 신고 및 업무시스템 구축
* 스톡옵션이란? 비상장 벤처기

--- chunk 1 len 800 ---
 조건을 충족해야 주식이 부여 대상자에게 귀속되도록 제한을 둔 주식
□ 복수의결권주식 발행 보고 및 스톡옵션 신고기능은 기존 시스템 내 벤
처확인 이력정보와 신고자(벤처기업)의 정보와 연동
- 3 -
2 사업개요
□ (사 업 명) 2024년 「벤처확인종합관리시스템 기능 고도화」 용역사업
- 복수의결권주식, 스톡옵션, 성과조건부주식교부 기능 구축 및 고도화
□ (사업기간) 계약일로부터 150일
* 1차 오픈 : 2024.07.10. 성과조건부 및 스톡옵션 오픈
** 2차 오픈 : 2024.09월 中 GrandOpen
□ (소요예산) 352,000,000원(부가가치세 포함)
□ (계약방식) 제한경쟁입찰(협상에 의한 계약)에 의한 선정*
* (근거) 「국가를 당사자로 하는 계약에 관한 법률」 시행령 제21조, 

### “깨짐/반복” 비율 체크

In [12]:
import re

def has_repeat_noise(t: str) -> bool:
    return re.search(r"(.)\1{3,}", t) is not None  # 같은 문자 4번 이상 반복

noisy = [i for i,c in enumerate(chunks) if has_repeat_noise(c)]
print("noisy_chunks:", len(noisy), "/", len(chunks), f"({len(noisy)/max(1,len(chunks)):.1%})")

# noisy 샘플 3개
for i in noisy[:3]:
    print("\n--- noisy chunk", i, "len", len(chunks[i]), "---")
    print(chunks[i][:500])

noisy_chunks: 7 / 130 (5.4%)

--- noisy chunk 11 len 800 ---
허위발행죄 및 과태료
중소벤처기업부는 규정에 위반한 혐의가 있을 경우 직권으로 조사, 누구든지 위반한 사실이
위반행위 인지‧신고
있다고 인정할 때에 중소벤처기업부에 신고
보고(변경보고 포함)하지 아니하거나, 보고내용 허위작성시 500만원 이하 과태료,
과태료
비치 및 공시의무 위반시 300만원 이하 과태료
허위 또는 부정한 방법으로 복수의결권 발행한 자는
허위발행죄
10년 이하 징역 또는 5천만원 이하 벌금
- 14 -
□ (비상장 벤처기업) 복수의결권주식 발행보고 前 비상장 벤처기업 여부
확인, 요건 충족 기업만 가능
◦ (비상장) 복수의결권주식 발행보고 기업 법인등록번호 조회
- 대기업기업집단 계열회사 편입 여부(데이터포털 연계)
◦ (벤처확인) 복수의결권주식 발행보고 기업 사업자등록번호, 법인등록번
호, 벤처확인발급번호 조회
- (벤처확인 프로세스) 벤처확인 재확인, 또는 취소 (청문절차), 재발급
등 복수의결권주식 발행 신고 가능 여부 체크
- 주주총회 결의 시 벤처확인

--- noisy chunk 12 len 800 ---
00001
확인정보 처리일 : 2024.02.15. -
정상
접수번호 : 20160410-S-00135
벤처확인발급번호 : 20160201
- 처리일 : 2016.04.10.
2016.02.01.~2018.01.31.
정상
① 벤처확인 이력 기준 복수의결권주식 발행보고, 스톡옵션 신고 이력이 있는 경우 노출
- 벤처확인 유효시작일~만료일 기간 내 정관개정을 통한 복수의결권주식 발행보고, 스톡옵션 신고 가
능
- 이상 : 대기업집단계열편입, 상장, 폐업/휴업 등이 있는 경우 표시
② 벤처확인이력 기준 복수의결권주식 발행보고는 ‘23.11.17부터 가능, 스톡옵션은 98년부터 신고 가능
- 사용자 활동이력은 업무시스템에 동일하게 노출되도록 구현
③ 벤처확인발급번호, 복수의결권주식 접수번호, 스톡옵션 접수번호 기준 출력 기

### “청킹/검색이 맞는지” anchor hit@k 빠른 점검 (bm25/dense/hybrid 비교)

In [13]:
sample_iids = (
    gold_fields_df[gold_fields_df["doc_id"] == doc_path.name]["instance_id"]
    .astype(str)
    .unique()
)

print("sample_iids:", sample_iids)
print("n_sample:", len(sample_iids))

sample_iids: <StringArray>
[   'G_Q001',    'G_Q002',    'G_Q003',    'G_Q004',    'G_Q005',    'G_Q006',
    'G_Q007',    'G_Q008',    'G_Q009',    'G_Q010',    'G_Q011', 'D001_X001',
 'D001_X002', 'D001_X003', 'D001_X004', 'D001_X005', 'D001_X006', 'D001_X007',
 'D001_X008', 'D001_X009', 'D001_X010']
Length: 21, dtype: str
n_sample: 21


In [14]:
import numpy as np

retrievers = ["bm25", "dense", "hybrid"]
hits = {r: [] for r in retrievers}
used_iids = 0

for iid in sample_iids:
    anchors = GOLD_ANCHOR.get((doc_path.name, str(iid)), [])
    if not anchors:
        print("\nIID:", iid, "(no anchors) skip")
        continue

    used_iids += 1
    q = questions_df.loc[questions_df["instance_id"].astype(str)==str(iid), "question"].iloc[0]

    print("\n============================")
    print("IID:", iid)
    print("Q:", q)
    print("anchors:", anchors)

    for r in retrievers:
        idxs = retrieve_indices(
            retriever=r,
            query_text=q,
            chunks=chunks,
            index=index,
            embed_model=embed_model,
            bm25=bm25,
            top_k=10,
            hybrid_cand_k=100,
        )
        hit = any(any(a in chunks[i] for a in anchors) for i in idxs)
        hits[r].append(float(hit))

        top1 = chunks[idxs[0]][:120].replace("\n"," ") if idxs else ""
        print(f"  {r:6s} hit@10:", hit, "top1:", top1)

print("\nused_iids:", used_iids)
for r in retrievers:
    if len(hits[r]) == 0:
        print(f"{r} hit@10: (no samples)")
    else:
        print(f"{r} hit@10:", float(np.mean(hits[r])))


IID: G_Q001
Q: 사업(용역)명은 무엇인가?
anchors: ['사업개요']
  bm25   hit@10: False top1: 및 매출액 현황 자본금 및 매출액 현황(최근 3년) (단위 : 천원) 구 분 M-2 년도 M-1 년도 M 년도 자 본 금 BPR/ISP 전략컨설팅 컨설팅부문 보안컨설팅 감리 매 출 액 기타 개발부문 교육부문 ⃝⃝부문
  dense  hit@10: True top1: 용역사업 - (복수의결권주식, 스톡옵션, 성과조건부주식) -   2024. 03. 목 차 1. 추진개요 3 2. 추진방안 5 3. 추진내용 9 4. 제안요청내용 24 5. 입찰관련사항 78 6. 제안서작성요령 82 
  hybrid hit@10: True top1: 및 매출액 현황 자본금 및 매출액 현황(최근 3년) (단위 : 천원) 구 분 M-2 년도 M-1 년도 M 년도 자 본 금 BPR/ISP 전략컨설팅 컨설팅부문 보안컨설팅 감리 매 출 액 기타 개발부문 교육부문 ⃝⃝부문

IID: G_Q002
Q: 발주 기관(수요기관)은 어디인가?
anchors: ['벤처기업확인기관']
  bm25   hit@10: True top1:  등) 및 평가대 상 사업의 특성에 따라 적용되는 관련 법 등을 참고하여 평가기준을 수립해야 함 - 75 - - 서비스 보안성 강화를 위해 운영서버의 개인정보 유출탐지 등 모니터 링, 개발서버 시험운영을 위한 개인정
  dense  hit@10: True top1: 가 발생하였 을 경우에는 연대하여 책임을 진다. 제14조 [운영위원회] ①공동수급체는 공동수급체의 구성원을 위원으로 하는 운영위원 회를 설치하여 계약이행에 관한 제반사항을 협의한다. ②이 협정서에 규정되지 아니한 사
  hybrid hit@10: True top1: 가 발생하였 을 경우에는 연대하여 책임을 진다. 제14조 [운영위원회] ①공동수급체는 공동수급체의 구성원을 위원으로 하는 운영위원 회를 설치하여 계약이행에 관한 제반사항을 협의한다. ②이 협정서에 규정되지 아니한 사

- used_iids: 20 → 샘플 20개에 대해 anchor가 존재해서 평가에 실제로 포함
- bm25 hit@10: 0.55 → 55%
- dense hit@10: 0.8 → 80%
- hybrid hit@10: 0.85 → 85%

### 시스템 설정

In [15]:
SYSTEM = (
    "너는 RFP(제안요청서) 문서에서 질문의 답을 '추출'하는 정보추출기다.\n"
    "문서에 근거가 없으면 추측하지 말고 빈 문자열로 둬라.\n"
    "절대 문서 원문을 길게 복사하지 말고, 핵심만 짧게 답하라.\n"
    "출력은 오직 'field<TAB>answer' 형식의 줄들만 허용한다.\n"
    "각 field는 반드시 정확히 한 줄씩 출력한다.\n"
)

### LLM 호출

In [16]:
from openai import OpenAI

client = OpenAI()

def run_generation(context: str, fields: list[str], llm_model: str, question_block: str) -> str:
    """
    context: 검색된 chunk 합친 텍스트
    fields: 출력해야 하는 field 리스트
    llm_model: "gpt-5-mini" or "gpt-5-nano"
    question_block: field별 질문 텍스트를 합친 것(예: build_query_prompt 결과)
    """
    format_rule = (
        "반드시 각 줄을 'field<TAB>answer' 형식으로만 출력하라.\n"
        "아래 field 목록에 있는 것만 field로 사용하라:\n"
        f"{fields}\n"
        "설명/불릿/추가 문장 금지.\n"
        "답이 없으면 answer를 빈 문자열로 두되, 각 field는 반드시 한 줄씩 출력하라.\n"
    )

    prompt = (
        f"{question_block}\n\n"
        f"[CONTEXT]\n{context}\n\n"
        f"{format_rule}"
    )

    resp = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt},
        ],
        max_completion_tokens=768,
    )
    return (resp.choices[0].message.content or "").strip()

### 파서(parse_field_answers_relaxed) 추가

In [17]:
def parse_field_answers_relaxed(text: str, fields: list[str]) -> dict[str, str]:
    """
    TSV(탭) 우선, 실패 시 콜론/대시/등호도 허용.
    field 목록에 없는 키는 무시.
    """
    m = {f: "" for f in fields}
    if not text:
        return m

    field_set = set(fields)

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        if "\t" in line:
            k, v = line.split("\t", 1)
        elif ":" in line:
            k, v = line.split(":", 1)
        elif " - " in line:
            k, v = line.split(" - ", 1)
        elif " = " in line:
            k, v = line.split(" = ", 1)
        else:
            continue

        k = k.strip().lstrip("-•").strip()
        v = v.strip()
        if k in field_set:
            m[k] = v

    return m

### 질문 블록 만들기 (question_block)

In [18]:
def build_question_block(queries: list[tuple[str, str]]) -> str:
    """
    queries: [(field, question), ...]
    """
    lines = ["[QUESTIONS]"]
    for f, q in queries:
        lines.append(f"{f}\t{q}")
    return "\n".join(lines)

### gold_fields_df → “문서별 gold dict”

In [19]:
def get_gold_for_instance(gold_fields_df, doc_id: str, instance_id: str) -> dict[str, str]:
    sub = gold_fields_df[
        (gold_fields_df["doc_id"] == doc_id) &
        (gold_fields_df["instance_id"].astype(str) == str(instance_id))
    ]
    return dict(zip(sub["field"].astype(str), sub["gold"].astype(str)))

### “질문 1개” 평가 함수

In [20]:
import numpy as np
from difflib import SequenceMatcher
import re

_ws = re.compile(r"\s+")

# 유사도/정규화
def norm_text(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip()
    s = _ws.sub(" ", s)
    return s

def text_similarity(a: str, b: str) -> float:
    a = norm_text(a)
    b = norm_text(b)
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a, b).ratio()

# Retrieval metric 계산
def retrieval_metrics(idxs, chunks, anchors, k: int):
    if not anchors:
        return np.nan, np.nan
    hit_pos = []
    for rank, i in enumerate(idxs[:k], start=1):
        for a in anchors:
            if a in chunks[i]:
                hit_pos.append(rank)
                break
    if not hit_pos:
        return 0.0, 0.0
    return 1.0, 1.0 / min(hit_pos)

# Generation metric 계산
def generation_metrics(pred: dict, gold: dict):
    fields = list(gold.keys())
    if not fields:
        return np.nan, np.nan, np.nan

    filled = 0
    matched = 0
    sims = []

    for f in fields:
        p = norm_text(pred.get(f, ""))
        g = norm_text(gold.get(f, ""))

        if p:
            filled += 1
        if p and g:
            sim = text_similarity(p, g)
            sims.append(sim)
            if sim >= 0.8:
                matched += 1

    fill_rate = filled / len(fields)
    match_rate = matched / len(fields)
    avg_sim = float(np.mean(sims)) if sims else 0.0
    return fill_rate, match_rate, avg_sim

# 컨텍스트 길이 자르기
def build_context_with_budget(chunks, idxs, max_chars: int = 1500) -> str:
    out = []
    used = 0
    for i in idxs:
        c = chunks[i].strip()
        if not c:
            continue
        # 구분자 포함 길이
        add_len = len(c) + (6 if out else 0)
        if used + add_len > max_chars:
            break
        out.append(c)
        used += add_len
    return "\n\n---\n\n".join(out)

# 단일 질문 평가
def evaluate_one_question(
    row,
    doc_id: str,
    retriever: str,
    llm_model: str,
    chunks,
    index,
    embed_model,
    bm25,
    GOLD_ANCHOR,
    gold_fields_df,
    k: int = 10,
):
    iid = str(row["instance_id"])
    q = str(row["question"])
    field = str(row["type"])  # questions.csv에서 type 컬럼이 field 역할

    anchors = GOLD_ANCHOR.get((doc_id, iid), [])

    idxs = retrieve_indices(
        retriever=retriever,
        query_text=q,
        chunks=chunks,
        index=index,
        embed_model=embed_model,
        bm25=bm25,
        top_k=k,
        hybrid_cand_k=200,
    )

    recall, mrr = retrieval_metrics(idxs, chunks, anchors, k)

    # context
    if not idxs:
        context = ""
    else:
        context = build_context_with_budget(chunks, idxs[:k], max_chars=1500)


    # gold / pred
    gold = get_gold_for_instance(gold_fields_df, doc_id, iid)
    fields = [field]  # 이 질문은 field 1개만 평가
    qblock = build_question_block([(field, q)])

    raw = safe_run_generation(context=context, fields=fields, llm_model=llm_model, question_block=qblock)
    pred = parse_field_answers_relaxed(raw, fields)

    fill, match, sim = generation_metrics(pred, gold)

    return {
        "doc_id": doc_id,
        "instance_id": iid,
        "field": field,
        "retrieval_recall@k": recall,
        "retrieval_mrr@k": mrr,
        "gen_fill_rate": fill,
        "gen_match_rate": match,
        "gen_avg_similarity": sim,
        "has_evidence": bool(anchors),
        "has_gold": bool(gold),
    }

# 예외 처리
def safe_run_generation(**kwargs):
    try:
        return run_generation(**kwargs)
    except Exception as e:
        print("[GEN_ERROR]", e)
        return ""  # 빈 응답

In [21]:
def evaluate_all_docs(
    questions_df: pd.DataFrame,
    gold_fields_df: pd.DataFrame,
    GOLD_ANCHOR: dict,
    embed_model,
    retrievers=("bm25", "dense", "hybrid"),
    generators=("gpt-5-mini", "gpt-5-nano"),
    k: int = 10,
):
    rows = []

    common_q = questions_df[questions_df["doc_id"] == "*"]
    doc_q = questions_df[questions_df["doc_id"] != "*"]

    for doc_id, qdf_doc in doc_q.groupby("doc_id"):
        doc_path = RAW_DIR / doc_id
        if not doc_path.exists():
            print("[SKIP] missing pdf:", doc_path)
            continue

        # 공통질문 + 개별질문 합치기
        if len(common_q) > 0:
            qdf_common = common_q.copy()
            qdf_common["doc_id"] = doc_id
            qdf = pd.concat([qdf_doc, qdf_common], ignore_index=True)
        else:
            qdf = qdf_doc

        # 문서별 chunks/index 재생성
        index, chunks = pp.gen_input(doc_path, embed_model)

        # 문서별 bm25 생성
        bm25 = build_bm25(chunks)

        for r in retrievers:
            for g in generators:
                per_q = []
                for _, row in qdf.iterrows():
                    per_q.append(
                        evaluate_one_question(
                            row=row,
                            doc_id=doc_id,
                            retriever=r,
                            llm_model=g,
                            chunks=chunks,
                            index=index,
                            embed_model=embed_model,
                            bm25=bm25,
                            GOLD_ANCHOR=GOLD_ANCHOR,
                            gold_fields_df=gold_fields_df,
                            k=k,
                        )
                    )

                df = pd.DataFrame(per_q)

                n_total = len(df)
                n_with_evidence = int(df["has_evidence"].sum())
                n_with_gold = int(df["has_gold"].sum())

                recall = float(df["retrieval_recall@k"].mean())
                mrr = float(df["retrieval_mrr@k"].mean())

                gdf = df[df["has_gold"]]
                fill = float(gdf["gen_fill_rate"].mean()) if len(gdf) else np.nan
                match = float(gdf["gen_match_rate"].mean()) if len(gdf) else np.nan
                sim = float(gdf["gen_avg_similarity"].mean()) if len(gdf) else np.nan

                score = (
                    0.35 * (0.0 if np.isnan(recall) else recall) +
                    0.15 * (0.0 if np.isnan(mrr) else mrr) +
                    0.25 * (0.0 if np.isnan(fill) else fill) +
                    0.15 * (0.0 if np.isnan(match) else match) +
                    0.10 * (0.0 if np.isnan(sim) else sim)
                )

                rows.append({
                    "doc_id": doc_id,
                    "retriever": r,
                    "generator": g,
                    "n_questions_total": float(n_total),
                    "n_questions_with_evidence": float(n_with_evidence),
                    "retrieval_recall@k": recall,
                    "retrieval_mrr@k": mrr,
                    "n_questions_with_gold_fields": float(n_with_gold),
                    "gen_fill_rate": fill,
                    "gen_match_rate": match,
                    "gen_avg_similarity": sim,
                    "score": float(score),
                })
                

    return pd.DataFrame(rows)

### retriever×generator 평균 테이블 만들기

In [22]:
summary_df = evaluate_all_docs(
    questions_df=questions_df,
    gold_fields_df=gold_fields_df,
    GOLD_ANCHOR=GOLD_ANCHOR,
    embed_model=embed_model,
    k=10
)
display(summary_df)

[GEN_ERROR] Error code: 400 - {'error': {'message': 'Could not finish the message because max_tokens or model output limit was reached. Please try again with higher max_tokens.', 'type': 'invalid_request_error', 'param': None, 'code': None}}
[GEN_ERROR] Error code: 400 - {'error': {'message': 'Could not finish the message because max_tokens or model output limit was reached. Please try again with higher max_tokens.', 'type': 'invalid_request_error', 'param': None, 'code': None}}
[GEN_ERROR] Error code: 400 - {'error': {'message': 'Could not finish the message because max_tokens or model output limit was reached. Please try again with higher max_tokens.', 'type': 'invalid_request_error', 'param': None, 'code': None}}
[GEN_ERROR] Error code: 400 - {'error': {'message': 'Could not finish the message because max_tokens or model output limit was reached. Please try again with higher max_tokens.', 'type': 'invalid_request_error', 'param': None, 'code': None}}
[GEN_ERROR] Error code: 400 - {'

,doc_id,retriever,generator,n_questions_total,n_questions_with_evidence,retrieval_recall@k,retrieval_mrr@k,n_questions_with_gold_fields,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
0,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,bm25,gpt-5-mini,21.0,20.0,0.55,0.360000,21.0,0.380952,0.095238,0.186999,0.374724
1,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,bm25,gpt-5-nano,21.0,20.0,0.55,0.360000,21.0,0.095238,0.047619,0.066667,0.284119
2,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,dense,gpt-5-mini,21.0,20.0,0.80,0.476389,21.0,0.619048,0.095238,0.267199,0.547226
3,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,dense,gpt-5-nano,21.0,20.0,0.80,0.476389,21.0,0.142857,0.047619,0.118043,0.406120
4,(사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf,hybrid,gpt-5-mini,21.0,20.0,0.80,0.476389,21.0,0.619048,0.095238,0.253812,0.545887
...,...,...,...,...,...,...,...,...,...,...,...,...
175,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,bm25,gpt-5-nano,21.0,20.0,0.80,0.544583,21.0,0.142857,0.095238,0.113565,0.423044
176,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,dense,gpt-5-mini,21.0,20.0,0.90,0.579306,21.0,0.619048,0.047619,0.297700,0.593571
177,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,dense,gpt-5-nano,21.0,20.0,0.90,0.579306,21.0,0.095238,0.095238,0.091156,0.449107
178,문화체육관광부 국립민속박물관_2024년 국립민속박물관 민속아카이브 자.pdf,hybrid,gpt-5-mini,21.0,20.0,0.90,0.579306,21.0,0.619048,0.095238,0.298676,0.600811


### 평균 계산

In [27]:
mean_cols = [
    "retrieval_recall@k",
    "retrieval_mrr@k",
    "gen_fill_rate",
    "gen_match_rate",
    "gen_avg_similarity",
    "score",
]

overall_mean = summary_df[mean_cols].mean().to_frame(name="overall_mean")
display(overall_mean)

,overall_mean
retrieval_recall@k,0.597686
retrieval_mrr@k,0.377409
gen_fill_rate,0.428307
gen_match_rate,0.089418
gen_avg_similarity,0.200445
score,0.406335


In [28]:
metrics = [
    "retrieval_recall@k",
    "retrieval_mrr@k",
    "gen_fill_rate",
    "gen_match_rate",
    "gen_avg_similarity",
    "score",
]

avg_by_exp = (
    summary_df
    .groupby(["retriever", "generator"])[metrics]
    .mean()
    .reset_index()
)

display(avg_by_exp)

,retriever,generator,retrieval_recall@k,retrieval_mrr@k,gen_fill_rate,gen_match_rate,gen_avg_similarity,score
0,bm25,gpt-5-mini,0.547945,0.346122,0.534921,0.085714,0.227704,0.413057
1,bm25,gpt-5-nano,0.547945,0.346122,0.207937,0.063492,0.118290,0.317036
2,dense,gpt-5-mini,0.622556,0.393053,0.644444,0.103175,0.272196,0.480660
3,dense,gpt-5-nano,0.622556,0.393053,0.260317,0.082540,0.147061,0.369019
4,hybrid,gpt-5-mini,0.622556,0.393053,0.641270,0.107937,0.275763,0.480937
5,hybrid,gpt-5-nano,0.622556,0.393053,0.280952,0.093651,0.161659,0.377304


---

### hybrid만 하기

In [23]:
"""
summary_df = evaluate_all_docs(
    questions_df=questions_df,
    gold_fields_df=gold_fields_df,
    GOLD_ANCHOR=GOLD_ANCHOR,
    embed_model=embed_model,
    retrievers=("hybrid",),
    generators=("gpt-5-mini",),
    k=10
)
"""

'\nsummary_df = evaluate_all_docs(\n    questions_df=questions_df,\n    gold_fields_df=gold_fields_df,\n    GOLD_ANCHOR=GOLD_ANCHOR,\n    embed_model=embed_model,\n    retrievers=("hybrid",),\n    generators=("gpt-5-mini",),\n    k=10\n)\n'

In [24]:
# display(summary_df)

### hybrid 평균

In [25]:
"""
mean_cols = [
    "retrieval_recall@k",
    "retrieval_mrr@k",
    "gen_fill_rate",
    "gen_match_rate",
    "gen_avg_similarity",
    "score",
]

overall_mean = summary_df[mean_cols].mean().to_frame(name="overall_mean")
display(overall_mean)
"""

'\nmean_cols = [\n    "retrieval_recall@k",\n    "retrieval_mrr@k",\n    "gen_fill_rate",\n    "gen_match_rate",\n    "gen_avg_similarity",\n    "score",\n]\n\noverall_mean = summary_df[mean_cols].mean().to_frame(name="overall_mean")\ndisplay(overall_mean)\n'

In [26]:
"""
retrieval_summary = summary_df[[
    "retrieval_recall@k",
    "retrieval_mrr@k"
]].mean()

generation_summary = summary_df[[
    "gen_fill_rate",
    "gen_match_rate",
    "gen_avg_similarity"
]].mean()

print("=== Retrieval 평균 ===")
display(retrieval_summary.to_frame("mean"))

print("=== Generation 평균 ===")
display(generation_summary.to_frame("mean"))
"""

'\nretrieval_summary = summary_df[[\n    "retrieval_recall@k",\n    "retrieval_mrr@k"\n]].mean()\n\ngeneration_summary = summary_df[[\n    "gen_fill_rate",\n    "gen_match_rate",\n    "gen_avg_similarity"\n]].mean()\n\nprint("=== Retrieval 평균 ===")\ndisplay(retrieval_summary.to_frame("mean"))\n\nprint("=== Generation 평균 ===")\ndisplay(generation_summary.to_frame("mean"))\n'